In [17]:
from src.helpers import *
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [18]:
# Reload the saved scaled data
X_train = pd.read_csv('data/train/scaled_X_train.csv', parse_dates=True, index_col='unix')
y_train = pd.read_csv('data/train/y_train.csv', parse_dates=True, index_col='unix')
X_test = pd.read_csv('data/test/scaled_X_test.csv', parse_dates=True, index_col='unix')
y_test = pd.read_csv('data/test/y_test.csv', parse_dates=True, index_col='unix')

In [19]:
vif_transform = VIFTransform()
X_train = vif_transform.fit_transform(X_train)

/home/boshlike/.local/share/virtualenvs/final_assignment_code-eTELY0YF/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [20]:
X = X_train.values
y = y_train.values
y = y.ravel()

In [21]:
# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)


In [22]:
# find all relevant features - 5 features should be selected
feat_selector.fit(X, y)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	79
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	79
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	79
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	79
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	79
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	79
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	79
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	11
Rejected: 	68
Iteration: 	9 / 100
Confirmed: 	4
Tentative: 	7
Rejected: 	68
Iteration: 	10 / 100
Confirmed: 	4
Tentative: 	7
Rejected: 	68
Iteration: 	11 / 100
Confirmed: 	4
Tentative: 	7
Rejected: 	68
Iteration: 	12 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	69
Iteration: 	13 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	69
Iteration: 	14 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	69
Iteration: 	15 / 100
Confirmed: 	4
Tentative: 	6
Rejected: 	69
Iteration: 	16 / 100
Confirmed: 	4
Tentative: 	5
Rejected: 	70


BorutaPy(estimator=RandomForestClassifier(class_weight='balanced', max_depth=5,
                                          n_estimators=74, n_jobs=-1,
                                          random_state=RandomState(MT19937) at 0x7598455CC240),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7598455CC240, verbose=2)

In [23]:
# check selected features - first 5 features are selected
print(len(feat_selector.support_))
feat_selector.support_

79


array([False, False, False,  True, False,  True, False, False,  True,
        True,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False])

In [24]:
# check ranking of features

feat_selector.ranking_


array([ 3, 11, 26,  1,  7,  1, 21,  2,  1,  1,  1, 21, 17, 16, 11,  6,  1,
        4, 25,  9,  7, 24, 11, 28, 15, 13, 23, 20, 14, 19, 18, 27, 31, 29,
       29,  4, 32, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53,
       53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53,
       53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53])

In [25]:
# call transform() on X to filter it down to selected features
columns = X_train.head(1000).columns[feat_selector.support_]
print(columns)
X_filtered = feat_selector.transform(X)
X_filtered

Index(['PIVOTS_TRAD_D_R1', 'PIVOTS_TRAD_D_S2', 'HA_open', 'SQZ_NO',
       'ABER_ZG_5_15', 'ICS_26'],
      dtype='object')


array([[ 0.04630013,  0.46864855,  0.40785987,  0.32097828, -0.65424313,
         0.83354067],
       [ 0.04609072,  0.47255201,  0.42357974,  0.33328769, -0.66942396,
         0.79406376],
       [ 0.03854134,  0.48806037,  0.53200943,  0.34920456, -0.13941615,
         0.8122594 ],
       ...,
       [ 0.57559685,  0.55980287,  0.50900545,  0.46810362,  0.04732475,
         0.23230594],
       [ 0.54626662,  0.5481026 ,  0.52228624,  0.45068731,  0.24654159,
         0.31604697],
       [ 0.65472826,  0.52684653,  0.50816601,  0.43370444,  0.31980374,
         0.17640634]])